In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HTML as WHTML  # ← ipywidgetsのHTMLウィジェット
import requests
import webbrowser
import os
from dotenv import load_dotenv


# == パラメータ ==
INDEX_FILE = "faiss_index.bin"
METADATA_NPZ = "faiss_metadata.npz"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMB_DIM = 384
TOP_K = 5

# == モデル読込 ==
model = SentenceTransformer(MODEL_NAME)

# == FAISSインデックスとデータの読込 ==
index = faiss.read_index(INDEX_FILE)
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data["metadata_list"].tolist()
paragraphs = data["paragraphs"].tolist()

print("インデックスとメタデータを読み込みました。ベクトル数:", index.ntotal)

# == 検索関数 ==
def search_faiss(query, top_k=TOP_K):
    emb_query = model.encode([query], show_progress_bar=False).astype("float32")
    emb_query /= np.linalg.norm(emb_query, axis=1, keepdims=True)

    D, I = index.search(emb_query, top_k)

    results = []
    for score, idx in zip(D[0], I[0]):
        item = metadata_list[idx]
        paragraph_text = paragraphs[idx]
        results.append({
            "score": float(score),
            "text": paragraph_text,
            "metadata": item,
            "idx": idx
        })
    return results

# == GUIパーツ ==
query_box = widgets.Text(description="Query:", layout=widgets.Layout(width='400px'))
search_button = widgets.Button(description="Search")
output_area = widgets.Output()

# == 検索処理 ==
def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_box.value.strip()
        if not query:
            display(HTML("<b>Please enter a query.</b>"))
            return

        results = search_faiss(query, top_k=TOP_K)
        display(HTML(f"<h3>Search Query: {query}</h3>"))

        for r in results:
            
            score = r["score"]
            text = r["text"]
            meta = r["metadata"]
            idx = r["idx"]

            # === Toggle areas for previous and next paragraphs ===
            prev_box = widgets.VBox()
            next_box = widgets.VBox()

            # === Previous paragraph ===
            if idx > 0:
                prev_paragraph = paragraphs[idx - 1]
                toggle_prev = widgets.ToggleButton(description="⬇️ Show Previous Paragraph")
                output_prev = widgets.Output()

                def on_toggle_prev(change, out=output_prev, content=prev_paragraph, button=toggle_prev):
                    if change["new"]:
                        with out:
                            out.clear_output()
                            display(HTML(f"<div style='white-space: pre-wrap;'>{content}</div>"))
                        button.description = "⬆️ Hide Previous Paragraph"
                    else:
                        out.clear_output()
                        button.description = "⬇️ Show Previous Paragraph"

                toggle_prev.observe(on_toggle_prev, names="value")
                prev_box.children = [toggle_prev, output_prev]

            # === Next paragraph ===
            if idx < len(paragraphs) - 1:
                next_paragraph = paragraphs[idx + 1]
                toggle_next = widgets.ToggleButton(description="⬇️ Show Next Paragraph")
                output_next = widgets.Output()

                def on_toggle_next(change, out=output_next, content=next_paragraph, button=toggle_next):
                    if change["new"]:
                        with out:
                            out.clear_output()
                            display(HTML(f"<div style='white-space: pre-wrap;'>{content}</div>"))
                        button.description = "⬆️ Hide Next Paragraph"
                    else:
                        out.clear_output()
                        button.description = "⬇️ Show Next Paragraph"

                toggle_next.observe(on_toggle_next, names="value")
                next_box.children = [toggle_next, output_next]

            # === Main paragraph display ===
            main_paragraph_html = WHTML(
                f"<b>Score:</b> {score:.4f}<br>"
                f"<b>ID:</b> {meta.get('id')}<br>"
                f"<b>Page:</b> {meta.get('page')}<br>"
                f"<b>Source:</b> {meta.get('source')}<br>"
                f"<div style='margin-top:5px; border:1px solid #ccc; padding:5px; white-space:pre-wrap;'>{text}</div>"
            )

            full_block = widgets.VBox([prev_box, main_paragraph_html, next_box])
            display(full_block)

            display(HTML("<hr>"))


# イベント接続
search_button.on_click(on_search_clicked)

# 表示
display(query_box, search_button, output_area)



インデックスとメタデータを読み込みました。ベクトル数: 35299


Text(value='', description='Query:', layout=Layout(width='400px'))

Button(description='Search', style=ButtonStyle())

Output()

In [3]:
#Zoteroで開くボタン付き
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HTML as WHTML
import requests
import webbrowser
import os
from dotenv import load_dotenv

# === Zotero APIの設定を読み込み ===
load_dotenv()
ZOTERO_USER_ID = os.getenv("ZOTERO_USER_ID")
ZOTERO_API_KEY = os.getenv("ZOTERO_API_KEY")

# == パラメータ ==
INDEX_FILE = "faiss_index.bin"
METADATA_NPZ = "faiss_metadata.npz"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMB_DIM = 384
TOP_K = 5

# == モデル読込 ==
model = SentenceTransformer(MODEL_NAME)

# == FAISSインデックスとデータの読込 ==
index = faiss.read_index(INDEX_FILE)
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data["metadata_list"].tolist()
paragraphs = data["paragraphs"].tolist()

print("インデックスとメタデータを読み込みました。ベクトル数:", index.ntotal)

# == FAISS検索関数 ==
def search_faiss(query, top_k=TOP_K):
    emb_query = model.encode([query], show_progress_bar=False).astype("float32")
    emb_query /= np.linalg.norm(emb_query, axis=1, keepdims=True)

    D, I = index.search(emb_query, top_k)

    results = []
    for score, idx in zip(D[0], I[0]):
        item = metadata_list[idx]
        paragraph_text = paragraphs[idx]
        results.append({
            "score": float(score),
            "text": paragraph_text,
            "metadata": item,
            "idx": idx
        })
    return results

# == GUIパーツ ==
query_box = widgets.Text(description="Query:", layout=widgets.Layout(width='400px'))
search_button = widgets.Button(description="Search")
output_area = widgets.Output()

# == 検索ボタンクリック時の処理 ==
def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_box.value.strip()
        if not query:
            display(HTML("<b>Please enter a query.</b>"))
            return

        results = search_faiss(query, top_k=TOP_K)
        display(HTML(f"<h3>Search Query: {query}</h3>"))

        for r in results:
            score = r["score"]
            text = r["text"]
            meta = r["metadata"]
            idx = r["idx"]

            # === 前後のパラグラフ表示 ===
            prev_box = widgets.VBox()
            next_box = widgets.VBox()

            if idx > 0:
                prev_paragraph = paragraphs[idx - 1]
                toggle_prev = widgets.ToggleButton(description="⬇️ Show Previous Paragraph")
                output_prev = widgets.Output()

                def on_toggle_prev(change, out=output_prev, content=prev_paragraph, button=toggle_prev):
                    if change["new"]:
                        with out:
                            out.clear_output()
                            display(HTML(f"<div style='white-space: pre-wrap;'>{content}</div>"))
                        button.description = "⬆️ Hide Previous Paragraph"
                    else:
                        out.clear_output()
                        button.description = "⬇️ Show Previous Paragraph"

                toggle_prev.observe(on_toggle_prev, names="value")
                prev_box.children = [toggle_prev, output_prev]

            if idx < len(paragraphs) - 1:
                next_paragraph = paragraphs[idx + 1]
                toggle_next = widgets.ToggleButton(description="⬇️ Show Next Paragraph")
                output_next = widgets.Output()

                def on_toggle_next(change, out=output_next, content=next_paragraph, button=toggle_next):
                    if change["new"]:
                        with out:
                            out.clear_output()
                            display(HTML(f"<div style='white-space: pre-wrap;'>{content}</div>"))
                        button.description = "⬆️ Hide Next Paragraph"
                    else:
                        out.clear_output()
                        button.description = "⬇️ Show Next Paragraph"

                toggle_next.observe(on_toggle_next, names="value")
                next_box.children = [toggle_next, output_next]

            # === メイン表示ブロック ===
            main_paragraph_html = WHTML(
                f"<b>Score:</b> {score:.4f}<br>"
                f"<b>ID:</b> {meta.get('id')}<br>"
                f"<b>Page:</b> {meta.get('page')}<br>"
                f"<b>Source:</b> {meta.get('source')}<br>"
                f"<div style='margin-top:5px; border:1px solid #ccc; padding:5px; white-space:pre-wrap;'>{text}</div>"
            )

            # === Zoteroで開くボタン ===
            def on_zotero_open_clicked(btn, full_path=meta.get("source")):
                if not full_path:
                    print("PDFファイルのパスが指定されていません。")
                    return

                # ファイル名だけを取り出す
                pdf_folder = os.getenv("PDF_FOLDER", "")
                if full_path.startswith(pdf_folder):
                    pdf_name = full_path.replace(pdf_folder, "").lstrip("/\\")
                else:
                    pdf_name = os.path.basename(full_path)  # 念のための保険

                url = f"https://api.zotero.org/users/{ZOTERO_USER_ID}/items"
                headers = {"Zotero-API-Key": ZOTERO_API_KEY}
                params = {"q": pdf_name, "itemType": "attachment", "format": "json"}

                res = requests.get(url, headers=headers, params=params)
                if res.status_code == 200:
                    try:
                        items = res.json()
                        for item in items:
                            if item["data"].get("title") == pdf_name:
                                parent_key = item["data"].get("parentItem")
                                if parent_key:
                                    webbrowser.open(f"zotero://select/items/0_{parent_key}")
                                    return
                        print(f"Zotero: 該当ファイルが見つかりません: {pdf_name}")
                    except Exception as e:
                        print("Zotero: JSON解析エラー:", e)
                else:
                    print(f"Zotero APIエラー: {res.status_code}")


            zotero_button = widgets.Button(description="ZoteroでPDFを開く")
            zotero_button.on_click(on_zotero_open_clicked)

            # === 結果の表示 ===
            full_block = widgets.VBox([prev_box, main_paragraph_html, zotero_button, next_box])
            display(full_block)
            display(HTML("<hr>"))

# イベント接続
search_button.on_click(on_search_clicked)

# 表示
display(query_box, search_button, output_area)


インデックスとメタデータを読み込みました。ベクトル数: 35299


Text(value='', description='Query:', layout=Layout(width='400px'))

Button(description='Search', style=ButtonStyle())

Output()

In [ ]:
#英語でクエリ入力・回答生成（gemini APIを使用）
import numpy as np
import faiss
import ipywidgets as widgets
from IPython.display import display, HTML
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()  # .envファイルを読み込む

API_KEY = os.getenv("GEMINI_API_KEY") # 環境変数からAPIキーを取得

# === 設定 ===  
INDEX_FILE = 'faiss_index.bin'
METADATA_NPZ = 'faiss_metadata.npz'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
EMB_DIM = 384
TOP_K = 5

# === Gemini初期化 ===
genai.configure(api_key=API_KEY)
# === モデル・インデックス・データロード ===
print("Loading model...")
model = SentenceTransformer(MODEL_NAME)

print("Loading FAISS index...")
index = faiss.read_index(INDEX_FILE)

print("Loading metadata...")
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data['metadata_list'].tolist()
paragraphs = data['paragraphs'].tolist()

# === 検索関数 ===
def search_faiss(query, k=TOP_K):
    q_vec = model.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, k)

    results = []
    for score, idx in zip(D[0], I[0]):
        para_text = paragraphs[idx]
        meta = metadata_list[idx]
        results.append({
            'score': float(score),
            'paragraph': para_text,
            'metadata': meta
        })
    return results

# === Geminiで回答生成（引用番号付き）===
def generate_answer_with_gemini(query, top_results):
    context_str_list = []
    for i, r in enumerate(top_results, 1):
        para_text = r['paragraph']
        context_str_list.append(f"[{i}] {para_text}")

    context_str = "\n\n".join(context_str_list)
    prompt = (
        f"Use the numbered context paragraphs below to answer the question concisely. "
        f"Reference the sources using [1], [2], etc., where appropriate.\n\n"
        f"{context_str}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )

    model_gemini = genai.GenerativeModel('models/gemini-1.5-flash-latest')
    response = model_gemini.generate_content(prompt)
    return response.text

# === GUI部分 ===
query_box = widgets.Text(
    description='Query:',
    layout=widgets.Layout(width='500px'),
    placeholder='Enter your question...'
)
search_button = widgets.Button(description="Search & Answer", button_style='success')
output_area = widgets.Output()

def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_box.value.strip()
        if not query:
            display(HTML("<b style='color:red;'>Please enter a query.</b>"))
            return
        
        print("Searching documents...")
        top_results = search_faiss(query, k=TOP_K)
        
        print("Generating answer with Gemini...")
        answer = generate_answer_with_gemini(query, top_results)

        # 結果表示
        display(HTML(f"<h3>Gemini Answer</h3><p>{answer}</p>"))

        # 引用文献表示
        html_refs = "<h4>References</h4><ol>"
        for i, r in enumerate(top_results, 1):
            meta = r['metadata']
            html_refs += (
                f"<li><b>{meta.get('title')}</b> (page {meta.get('page')})<br>"
                f"<i>{meta.get('source')}</i></li>"
            )
        html_refs += "</ol>"
        display(HTML(html_refs))

search_button.on_click(on_search_clicked)

# === 表示 ===
display(widgets.VBox([query_box, search_button, output_area]))


In [ ]:
#日本語対応（クエリの翻訳にもgeminiを使用）
import numpy as np
import faiss
import ipywidgets as widgets
from IPython.display import display, HTML
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from dotenv import load_dotenv
import os

# === 環境変数の読み込み ===
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

# === Gemini初期化 ===
genai.configure(api_key=API_KEY)

# === 設定 ===
INDEX_FILE = 'faiss_index.bin'
METADATA_NPZ = 'faiss_metadata.npz'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
EMB_DIM = 384
TOP_K = 5

# === モデル・インデックス・メタデータの読み込み ===
print("Loading model...")
model = SentenceTransformer(MODEL_NAME)

print("Loading FAISS index...")
index = faiss.read_index(INDEX_FILE)

print("Loading metadata...")
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data['metadata_list'].tolist()
paragraphs = data['paragraphs'].tolist()

# === Geminiで日本語→英語 翻訳 ===
def translate_query_to_english(japanese_query):
    prompt = f"次の日本語を正確な英語に翻訳してください：\n\n{japanese_query}"
    translator = genai.GenerativeModel('models/gemini-1.5-flash-latest')
    response = translator.generate_content(prompt)
    return response.text.strip()

# === FAISS検索関数 ===
def search_faiss(query, k=TOP_K):
    q_vec = model.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, k)

    results = []
    for score, idx in zip(D[0], I[0]):
        para_text = paragraphs[idx]
        meta = metadata_list[idx]
        results.append({
            'score': float(score),
            'paragraph': para_text,
            'metadata': meta
        })
    return results

# === Geminiで回答生成（日本語出力・出典番号付き）===
def generate_answer_with_gemini(original_japanese_query, top_results):
    context_str_list = []
    for i, r in enumerate(top_results, 1):
        para_text = r['paragraph']
        context_str_list.append(f"[{i}] {para_text}")

    context_str = "\n\n".join(context_str_list)
    prompt = (
        f"以下の番号付きの文脈を参考にして、質問に日本語で簡潔に答えてください。\n"
        f"適切な箇所には [1], [2] などで出典を示してください。\n\n"
        f"{context_str}\n\n"
        f"質問: {original_japanese_query}\n\n"
        f"回答:"
    )

    model_gemini = genai.GenerativeModel('models/gemini-1.5-flash-latest')
    response = model_gemini.generate_content(prompt)
    return response.text

# === GUI作成 ===
query_box = widgets.Text(
    description='質問:',
    layout=widgets.Layout(width='500px'),
    placeholder='日本語で質問を入力してください...'
)
search_button = widgets.Button(description="検索 & 回答生成", button_style='success')
output_area = widgets.Output()

def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        japanese_query = query_box.value.strip()
        if not japanese_query:
            display(HTML("<b style='color:red;'>質問を入力してください。</b>"))
            return

        print("英語に翻訳中...")
        english_query = translate_query_to_english(japanese_query)

        print("FAISSで検索中...")
        top_results = search_faiss(english_query, k=TOP_K)

        print("Geminiで回答生成中...")
        answer = generate_answer_with_gemini(japanese_query, top_results)

        # 回答表示
        display(HTML(f"<h3>Geminiの回答</h3><p>{answer}</p>"))

        # 引用文献表示
        html_refs = "<h4>出典</h4><ol>"
        for i, r in enumerate(top_results, 1):
            meta = r['metadata']
            html_refs += (
                f"<li><b>{meta.get('title')}</b> (page {meta.get('page')})<br>"
                f"<i>{meta.get('source')}</i></li>"
            )
        html_refs += "</ol>"
        display(HTML(html_refs))

search_button.on_click(on_search_clicked)

# === 表示 ===
display(widgets.VBox([query_box, search_button, output_area]))


In [ ]:
#Mistral 7Bを使用（ollama経由）
import numpy as np
import faiss
import ipywidgets as widgets
from IPython.display import display, HTML
from sentence_transformers import SentenceTransformer
import subprocess

# === 設定 ===  
INDEX_FILE = 'faiss_index.bin'
METADATA_NPZ = 'faiss_metadata.npz'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
EMB_DIM = 384
TOP_K = 5

print("Loading model...")
model = SentenceTransformer(MODEL_NAME)

print("Loading FAISS index...")
index = faiss.read_index(INDEX_FILE)

print("Loading metadata...")
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data['metadata_list'].tolist()
paragraphs = data['paragraphs'].tolist()

# === 検索関数 ===
def search_faiss(query, k=TOP_K):
    q_vec = model.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, k)

    results = []
    for score, idx in zip(D[0], I[0]):
        para_text = paragraphs[idx]
        meta = metadata_list[idx]
        results.append({
            'score': float(score),
            'paragraph': para_text,
            'metadata': meta
        })
    return results

# === mistralで回答生成（引用番号付き）===
def generate_answer_with_mistral(query, top_results):
    context_str_list = []
    for i, r in enumerate(top_results, 1):
        para_text = r['paragraph']
        context_str_list.append(f"[{i}] {para_text}")
        
    context_str = "\n\n".join(context_str_list)
    prompt = (
        f"Use the numbered context paragraphs below to answer the question concisely. "
        f"Reference the sources using [1], [2], etc., where appropriate.\n\n"
        f"{context_str}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )

    # ここでは標準入力経由でプロンプトを渡す
    # ollama のバージョンに応じてコマンドが異なる場合があります
    command = ["ollama", "run", "mistral:latest"]
    result = subprocess.run(
        command,
        input=prompt,          # ← プロンプトを標準入力として渡す
        text=True,
        capture_output=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"Ollama command failed: {result.stderr}")

    return result.stdout.strip()

# === GUI部分 ===
query_box = widgets.Text(
    description='Query:',
    layout=widgets.Layout(width='500px'),
    placeholder='Enter your question...'
)
search_button = widgets.Button(description="Search & Answer", button_style='success')
output_area = widgets.Output()

def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_box.value.strip()
        if not query:
            display(HTML("<b style='color:red;'>Please enter a query.</b>"))
            return
        
        print("Searching documents...")
        top_results = search_faiss(query, k=TOP_K)
        
        print("Generating answer with Mistral via Ollama...")
        answer = generate_answer_with_mistral(query, top_results)

        # 結果表示
        display(HTML(f"<h3>Mistral Answer</h3><p>{answer}</p>"))

        # 引用文献表示
        html_refs = "<h4>References</h4><ol>"
        for i, r in enumerate(top_results, 1):
            meta = r['metadata']
            html_refs += (
                f"<li><b>{meta.get('title')}</b> (page {meta.get('page')})<br>"
                f"<i>{meta.get('source')}</i></li>"
            )
        html_refs += "</ol>"
        display(HTML(html_refs))

search_button.on_click(on_search_clicked)

# === 表示 ===
display(widgets.VBox([query_box, search_button, output_area]))



In [ ]:
# phiを使用（ollama経由）
import numpy as np
import faiss
import ipywidgets as widgets
from IPython.display import display, HTML
from sentence_transformers import SentenceTransformer
import subprocess

# === 設定 ===  
INDEX_FILE = 'faiss_index.bin'
METADATA_NPZ = 'faiss_metadata.npz'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
EMB_DIM = 384
TOP_K = 5

print("Loading model...")
model = SentenceTransformer(MODEL_NAME)

print("Loading FAISS index...")
index = faiss.read_index(INDEX_FILE)

print("Loading metadata...")
data = np.load(METADATA_NPZ, allow_pickle=True)
metadata_list = data['metadata_list'].tolist()
paragraphs = data['paragraphs'].tolist()

# === 検索関数 ===
def search_faiss(query, k=TOP_K):
    q_vec = model.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, k)

    results = []
    for score, idx in zip(D[0], I[0]):
        para_text = paragraphs[idx]
        meta = metadata_list[idx]
        results.append({
            'score': float(score),
            'paragraph': para_text,
            'metadata': meta
        })
    return results

# === phiで回答生成（引用番号付き）===
def generate_answer_with_phi(query, top_results):
    context_str_list = []
    for i, r in enumerate(top_results, 1):
        para_text = r['paragraph']
        context_str_list.append(f"[{i}] {para_text}")
        
    context_str = "\n\n".join(context_str_list)
    prompt = (
        f"Use the numbered context paragraphs below to answer the question concisely. "
        f"Reference the sources using [1], [2], etc., where appropriate.\n\n"
        f"{context_str}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )

    # ここでは標準入力経由でプロンプトを渡す
    command = ["ollama", "run", "phi:latest"]
    result = subprocess.run(
        command,
        input=prompt,
        text=True,
        capture_output=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"Ollama command failed: {result.stderr}")

    return result.stdout.strip()

# === GUI部分 ===
query_box = widgets.Text(
    description='Query:',
    layout=widgets.Layout(width='500px'),
    placeholder='Enter your question...'
)
search_button = widgets.Button(description="Search & Answer", button_style='success')
output_area = widgets.Output()

def on_search_clicked(b):
    with output_area:
        output_area.clear_output()
        query = query_box.value.strip()
        if not query:
            display(HTML("<b style='color:red;'>Please enter a query.</b>"))
            return
        
        print("Searching documents...")
        top_results = search_faiss(query, k=TOP_K)
        
        print("Generating answer with phi via Ollama...")
        answer = generate_answer_with_phi(query, top_results)

        # 結果表示
        display(HTML(f"<h3>phi Answer</h3><p>{answer}</p>"))

        # 引用文献表示
        html_refs = "<h4>References</h4><ol>"
        for i, r in enumerate(top_results, 1):
            meta = r['metadata']
            html_refs += (
                f"<li><b>{meta.get('title')}</b> (page {meta.get('page')})<br>"
                f"<i>{meta.get('source')}</i></li>"
            )
        html_refs += "</ol>"
        display(HTML(html_refs))

search_button.on_click(on_search_clicked)

# === 表示 ===
display(widgets.VBox([query_box, search_button, output_area]))
